# Setup

In [2]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import nltk
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/irene.gaita/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/irene.gaita/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/irene.gaita/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# Preprocessing Dataset Originale

In [3]:
# Caricare il dataset
df = pd.read_csv("../sentiment_data/all-dataset.csv", encoding='latin1', names=['label', 'text'])

# Rimuovere le stopwords
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

# Funzione per il preprocessing del testo
def preprocess_text(text):
    if not text:
        return ""

    # Convertire tutto il testo in minuscolo
    text = text.lower()

    # Rimuovere i numeri
    text = re.sub(r'\d+', '', text)

    # Rimuovere la punteggiatura
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenizzare il testo
    words = nltk.word_tokenize(text)

    # Rimuovere le stopwords
    words = [word for word in words if word not in stop_words]

    # Stemming
    words = [stemmer.stem(word) for word in words]

    # Lemmatizzazione
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Restituire il testo preprocessato
    return ' '.join(words)

# Applicare il preprocessing al dataset
df['cleaned_text'] = df['text'].apply(preprocess_text)

# Aggiungere la lunghezza delle frasi originali e preprocessate
df['length_original'] = df['text'].apply(lambda x: len(x.split()))
df['length_preprocessed'] = df['cleaned_text'].apply(lambda x: len(x.split()))

# Soglia per le lunghezze estreme (frasi con meno di o uguali a 2 parole)
soglia_lunghezza = 2

# Filtrare le frasi con lunghezza inferiore o uguale alla soglia
df = df[(df['length_original'] > soglia_lunghezza) & (df['length_preprocessed'] > soglia_lunghezza)]

# Rimuovere le colonne 'length_original' e 'length_preprocessed'
df = df.drop(columns=['length_original', 'length_preprocessed'])

# Visualizzare il numero di righe nel dataframe filtrato
numero_righe = df.shape[0]
print(f"Numero di righe nel dataframe filtrato: {numero_righe}")

# Esportare il dataframe filtrato (opzionale, se necessario)
df.to_csv("../sentiment_data/preprocessed_data.csv", index=False)

# Visualizzare le prime righe del dataframe filtrato
print("\nDataframe filtrato (senza frasi <= 2 parole):")
print(df.head())


Numero di righe nel dataframe filtrato: 4816

Dataframe filtrato (senza frasi <= 2 parole):
      label                                               text  \
0   neutral  According to Gran , the company has no plans t...   
1   neutral  Technopolis plans to develop in stages an area...   
2  negative  The international electronic industry company ...   
3  positive  With the new production plant the company woul...   
4  positive  According to the company 's updated strategy f...   

                                        cleaned_text  
0  accord gran compani plan move product russia a...  
1  technopoli plan develop stage area less squar ...  
2  intern electron industri compani elcoteq laid ...  
3  new product plant compani would increas capac ...  
4  accord compani updat strategi year baswar targ...  


# Preprocessing del Testo per la Sentiment Analysis

Il preprocessing del testo è un passaggio fondamentale per migliorare la performance dei modelli di machine learning, in quanto aiuta a ridurre la variabilità e a concentrare il modello sulle caratteristiche più rilevanti del testo.

### Passaggi di Preprocessing

1. **Caricamento del Dataset**
   - Il dataset è stato caricato utilizzando la libreria `pandas`:
     Il file CSV contiene due colonne: `label` (indicante il sentiment) e `text` (contenente i testi da analizzare).

2. **Rimozione delle Stopwords**
   - Le **stopwords** (parole comuni come "the", "and", "is") sono state rimosse dal testo. Queste parole non forniscono informazioni significative per la sentiment analysis.
     - Utilizzo della libreria `nltk` per ottenere la lista di stopwords in inglese:
       ```python
       stop_words = set(stopwords.words('english'))
       ```

3. **Funzione di Preprocessing**
   - È stata definita una funzione di preprocessing per elaborare ogni testo nel dataset:
     La funzione esegue le seguenti operazioni:

     - **Conversione in minuscolo**: Tutto il testo viene convertito in minuscolo per ridurre la variabilità causata dalla maiuscola/minuscola.
     - **Rimozione dei numeri**: I numeri vengono rimossi, poiché non sono considerati rilevanti per la sentiment analysis.
       ```python
       text = re.sub(r'\d+', '', text)
       ```
     - **Rimozione della punteggiatura**: I caratteri di punteggiatura (come punti, virgole, ecc.) vengono eliminati, poiché non aggiungono valore al sentiment.
       ```python
       text = re.sub(r'[^\w\s]', '', text)
       ```
     - **Tokenizzazione**: Il testo viene suddiviso in parole separate utilizzando la libreria `nltk`.
       ```python
       words = nltk.word_tokenize(text)
       ```
     - **Rimozione delle stopwords**: Le parole comuni (stopwords) vengono eliminate.
     - **Stamming e Lemmatizzazione**: Ogni parola viene ridotta alla sua forma base (lemma) utilizzando il lemmatizzatore `WordNetLemmatizer` della libreria `nltk`. Questo aiuta a ridurre la variabilità delle parole (es. "running" diventa "run").
     - **Eliminazione delle frasi con una lunghezza inferiore o uguale a 2 parole**: Frasi molto corte, come "welcome", "cloud", "thik", o singole parole, spesso non offrono un contesto sufficiente per un'analisi semantica accurata. Queste frasi non contribuiscono in modo significativo alla comprensione del sentiment e potrebbero introdurre rumore nel modello di machine learning.

4. **Applicazione del Preprocessing al Dataset**
   - La funzione di preprocessing è stata applicata a ogni riga del dataset nella colonna `text`, creando una nuova colonna `cleaned_text` che contiene il testo preprocessato.